In [1]:
%load_ext autoreload
%autoreload 2

## KEGGapFiller Example

In [1]:
from refinegems.classes.gapfill import KEGGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

print(len(model.getPlugin(0).getListOfGeneProducts()))

gfk = KEGGapFiller('sha')

gfk_missing_genes = gfk.get_missing_genes(model)

gfk_missing_reac = gfk.get_missing_reacs(cmodel,gfk_missing_genes)

model = gfk.fill_model(model,gfk_missing_genes,gfk_missing_reac)

print(len(model.getPlugin(0).getListOfGeneProducts()))

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


787
Running in debugging mode.


100%|██████████| 20/20 [00:24<00:00,  1.23s/it]


fill_model: Running in debugging mode


Adding genes to model: 100%|██████████| 2/2 [00:00<00:00, 3636.15it/s]

789


In [2]:
gfk._statistics 

{'genes': {'missing (before)': 20, 'added': 2, 'missing (after)': 18},
 'reactions': {'added (total)': 7, 'failed to build': 3},
 'metabolites': {}}

## GeneGapFiller Example

In [1]:
from refinegems.classes.gapfill import GeneGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_RefSeq.gff'

tfasta = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_proteins_genome.fasta'
spdb = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/swissprot.dmnd'
spmap = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/uniprot_table.tsv'
kwargs = {'outdir':'/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435',
          'sens':'more-sensitive',
          'cov':90.0,
          't':4,
          'pid':90.0}

gf2 = GeneGapFiller()
gf2_missing_genes = gf2.get_missing_genes(gffpath,model)
# ncbiprotein | locus_tag | ec-code
mapped_res = gf2.get_missing_reacs(model=cmodel,
                                   missing_genes=gf2_missing_genes,
                                   fasta=tfasta, 
                                   dmnd_db=spdb,
                                   swissprot_map=spmap,
                                   **kwargs)

model = gf2.fill_model(model,mapped_res[0],mapped_res[1],formula_check='existence')
# formula_check,exclude_dna,exclude_rna,idprefix,namespace

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


Running in debugging mode.
fill_model: Running in debugging mode


Adding genes to model: 100%|██████████| 3/3 [00:00<00:00, 5345.33it/s]


In [2]:
gf2._statistics
# possible problem with the current stats: some stuff might be counted twice or more, since the
# current implementation depends on the tables (esp. reactions, since ID depends on NCBI 
# protein accession number and not the uniqueness of the ID)


{'genes': {'missing (before)': 1800,
  'added': 3,
  'missing (after)': 4,
  'no locus tag': 108},
 'reactions': {'added (total)': 7,
  'failed to build': 3,
  'no NCBI, no EC': 0,
  'NCBI, no EC': 43},
 'metabolites': {}}

## BioCycGapFiller example

In [1]:
from refinegems.classes.gapfill import BioCycGapFiller
from refinegems.curation.db_access import biocyc
from refinegems.utility.io import load_model

modelpath = './test_files/ATCC29970.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/data/ATCC29970/ATCC29970_RefSeq.gff'
biocyc_gene_tbl_path = '/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/archive/master_thesis_ishgog23/data/ATCC29970/ATCC29970_Accession-22Reactions.txt'
biocyc_reacs_tbl_path = './test_files/ATCC29970_biocyc_rxns.txt'
fasta = '/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/data/ATCC29970/ATCC29970_proteins_genome.fasta'

gfbc = BioCycGapFiller(biocyc_gene_tbl_path, biocyc_reacs_tbl_path, gffpath)
gfbc_missing_genes = gfbc.get_missing_genes(model)
# ncbiprotein | locus_tag | ec-code
mapped_res = gfbc.get_missing_reacs(cmodel)

#model = gfbc.fill_model(model,mapped_res[0],mapped_res[1],formula_check='existence')
# formula_check,exclude_dna,exclude_rna,idprefix,namespace

/Users/doebel/miniconda3/envs/refineGEMs/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [2]:
gfbc.missing_reacs

,id,ncbiprotein,equation,ec-code,via,add_to_GPR,references
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN,[WP_011275497.1],acyl-[acyl-carrier protein] + a 2-lysophosphat...,[EC-2.3.1.51],BioCyc,None,"{'Reactants': ['ACYL-ACP', 'ACYL-SN-GLYCEROL-3..."
1,1.5.1.11-RXN,[WP_011275081.1],D-octopine + NAD+ + H2O = L-arginine + pyruv...,[EC-1.5.1.11],BioCyc,None,"{'Reactants': ['CPD-309', 'NAD', 'WATER'], 'Pr..."
2,1.8.1.4-RXN,[WP_037558251.1],a [lipoyl-carrier protein]-N6-[(R)-dihydrolipo...,[EC-1.8.1.4],BioCyc,None,"{'Reactants': ['Dihydro-Lipoyl-Proteins', 'NAD..."
3,2-HALOACID-DEHALOGENASE-RXN,"[WP_037559480.1, WP_011274778.1]",an (S)-2-halocarboxylate + H2O -> a (2R)-2-h...,[EC-3.8.1.10],BioCyc,None,"{'Reactants': ['S-2-Haloacids', 'WATER'], 'Pro..."
4,2-HALOACID-DEHALOGENASE-RXN,"[WP_037559480.1, WP_011274778.1]",an (S)-2-halocarboxylate + H2O -> a (2R)-2-h...,[EC-3.8.1.10],BioCyc,None,"{'Reactants': ['S-2-Haloacids', 'WATER'], 'Pro..."
...,...,...,...,...,...,...,...
272,TRYPTOPHAN--TRNA-LIGASE-RXN,[WP_011276231.1],L-tryptophan + tRNATrp + ATP -> AMP + L-tryp...,[EC-6.1.1.2],BioCyc,None,"{'Reactants': ['TRP', 'TRP-tRNAs', 'ATP'], 'Pr..."
273,TYROSINE--TRNA-LIGASE-RXN,[WP_011275493.1],tRNATyr + L-tyrosine + ATP -> L-tyrosyl-[tRN...,[EC-6.1.1.1],BioCyc,None,"{'Reactants': ['TYR-tRNAs', 'TYR', 'ATP'], 'Pr..."
274,UDP-NACMURALGLDAPAALIG-RXN,[WP_011275264.1],UDP-<I>N</I>-acetylmuramoyl-L-alanyl-D-glutamy...,[EC-6.3.2.10],BioCyc,None,{'Reactants': ['UDP-AAGM-DIAMINOHEPTANEDIOATE'...
275,UDPGLCNACEPIM-RXN,[WP_033079611.1],UDP-N-acetyl-alpha-D-glucosamine -> UDP-N-ac...,[EC-5.1.3.14],BioCyc,None,"{'Reactants': ['UDP-N-ACETYL-D-GLUCOSAMINE'], ..."


In [3]:
gfbc.missing_reacs.to_csv('./test_files/missing_reacs_test.tsv', sep='\t')

In [17]:
gfbc.missing_genes.to_csv('./test_files/missing_genes_gff.tsv', sep='\t')

#### Get new BioCyc reactions table without BioCyc access

In [16]:
import pandas as pd

df_with_new_info = pd.read_csv('./test_files/All-reactions-of-S.-haemolyticus-ATCC-29970.txt', sep='\t')
df_with_new_info.drop(['Object ID.1', 'EC-Number', 'Products of reaction', 'Pathways of a reaction'], axis=1, inplace=True)
df_old = pd.read_csv('/Users/doebel/Desktop/Master_thesis/Repository/Shaemolyticus/archive/master_thesis_ishgog23/data/ATCC29970/ATCC29970_reactions.txt', sep='\t')
df_old.rename({'Reaction': 'Object ID'}, axis=1, inplace=True)
df_old.drop('Accession-2', axis=1, inplace=True)

df_new = df_with_new_info.merge(df_old, how='left', on='Object ID')
df_new.to_csv('./test_files/ATCC29970_biocyc_rxns.txt', index=False, sep='\t')
df_new

,Reaction,Object ID,Substrates (reactants and products) of a reaction,Reactants of reaction,Products of reaction,EC-Number,Reaction-Direction,Spontaneous?
0,S-adenosyl-L-methionine + demethylmenaquinol-7...,RXN-9191,demethylmenaquinol-7 // <i>S</i>-adenosyl-L-me...,S-ADENOSYLMETHIONINE // CPD-12117,CPD-12125 // ADENOSYL-HOMO-CYS // PROTON,EC-2.1.1.163,PHYSIOL-LEFT-TO-RIGHT,NaN
1,"a 2,3,4-saturated (3R)-3-hydroxyacyl-CoA <-->...",OHBUTYRYL-COA-EPIM-RXN,"a 2,3,4-saturated (3<i>R</i>)-3-hydroxyacyl-Co...",D-3-HYDROXYACYL-COA,L-3-HYDROXYACYL-COA,EC-5.1.2.3,REVERSIBLE,NaN
2,"glycine + a tetrahydrofolate + NAD+ -> a 5,1...",GCVMULTI-RXN,NAD<sup>+</sup> // a tetrahydrofolate // glyci...,GLY // THF-GLU-N // NAD,METHYLENE-THF-GLU-N // AMMONIUM // CARBON-DIOX...,EC-1.4.1,LEFT-TO-RIGHT,NaN
3,N-acetyl-beta-neuraminate -> N-acetyl-beta-D...,RXN-15313,<i>N</i>-acetyl-&beta;-neuraminate // <i>N</i>...,CPD0-1123,N-ACETYL-D-MANNOSAMINE // PYRUVATE,EC-4.1.3.3,PHYSIOL-LEFT-TO-RIGHT,NaN
4,D-xylulose + ATP -> D-xylulose 5-phosphate +...,XYLULOKIN-RXN,ATP // D-xylulose // H<SUP>+</SUP> // D-xylulo...,D-XYLULOSE // ATP,PROTON // XYLULOSE-5-PHOSPHATE // ADP,EC-2.7.1.17,LEFT-TO-RIGHT,NaN
...,...,...,...,...,...,...,...,...
1064,(3R)-3-hydroxydecanoyl-[acp] -> (2E)-dec-2-e...,RXN-9655,a (3<i>R</i>)-3-hydroxydecanoyl-[acp] // a (2<...,Beta-hydroxydecanoyl-ACPs,Trans-D2-decenoyl-ACPs // WATER,EC-4.2.1.59,LEFT-TO-RIGHT,NaN
1065,2 L-arginine + 3 NADPH + H+ + 4 oxygen -> 2 ...,NITRIC-OXIDE-SYNTHASE-RXN,oxygen // H<SUP>+</SUP> // NADPH // L-arginine...,ARG // NADPH // PROTON // OXYGEN-MOLECULE,L-CITRULLINE // NITRIC-OXIDE // NADP // WATER,EC-1.14.13.39,PHYSIOL-LEFT-TO-RIGHT,NaN
1066,a [protein]-Npi-phospho-L-histidine + 2-phosph...,RXN-15510,2-phospho-D-glycerate // a [protein]-<i>N<sup>...,Protein-pi-phospho-L-histidines // 2-PG,Protein-Histidines // 23-DIPHOSPHOGLYCERATE,NaN,REVERSIBLE,NaN
1067,(5Z)-tetradecenoyl-CoA + acetyl-CoA <- 3-oxo...,RXN-17782,acetyl-CoA // (5Z)-tetradecenoyl-CoA // 3-oxo-...,CPD-19167 // CO-A,CPD-15436 // ACETYL-COA,EC-2.3.1.16,PHYSIOL-RIGHT-TO-LEFT,NaN


## the "Filling" part of Gapfilling

**input**

- the model
- missing genes table
- missing reacs table

**out**

- the extended model

**else**

- logging
- save stats information

### Further ideas and Code snippets for the filling part

missing stuff:
- parse BioCyc reaction string
- build biocyc_metabolite
- build biocyc_reaction

### Ideas for collect the stats information

In [7]:
len(cmodel.reactions)

1538